In [1]:
import torch
import torchvision
import numpy as np
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual

batch_size_slider = widgets.IntSlider(value=20, min=10, max=100, description='Batch size:')
hl_slider1 = widgets.IntSlider(value=512, min=1, max=1000, description='Size layer 1:')
hl_slider2 = widgets.IntSlider(value=256, min=1, max=1000, description='Size layer 2:')
hl_slider3 = widgets.IntSlider(value=64, min=1, max=1000, description='Size layer 3:')
dropout_slider = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, description='Dropout prob:')
lr_slider = widgets.FloatLogSlider(value=0.01, base=10, min=-4, max=0, step=0.0001,description='Learning rate:')
epoch_slider = widgets.IntSlider(value=10, min=0, max=100, description='No of epochs:')



display(batch_size_slider)
display(hl_slider1)
display(hl_slider2)
display(hl_slider3)
display(dropout_slider)
display(lr_slider)
display(epoch_slider)

IntSlider(value=20, description='Batch size:', min=10)

IntSlider(value=512, description='Size layer 1:', max=1000, min=1)

IntSlider(value=256, description='Size layer 2:', max=1000, min=1)

IntSlider(value=64, description='Size layer 3:', max=1000, min=1)

FloatSlider(value=0.2, description='Dropout prob:', max=1.0)

FloatLogSlider(value=0.01, description='Learning rate:', max=0.0, min=-4.0, step=0.0001)

IntSlider(value=10, description='No of epochs:')

In [2]:
def init_data():
    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

    train_data = torchvision.datasets.MNIST(root='data', download=True, train=True, transform = transform)
    test_data = torchvision.datasets.MNIST(root='data', download=True, train=False, transform = transform)

    #Obtain train-validation split
    indices = list(range(len(train_data)))
    np.random.shuffle(indices)

    BATCH_SIZE = batch_size_slider.value
    validation_ratio = 0.2
    split_point = int(np.floor(len(indices)*validation_ratio))
    validation_indices = indices[:split_point]
    train_indices = indices[split_point:]
    
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
    validation_sampler = torch.utils.data.sampler.SubsetRandomSampler(validation_indices)

    train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
    validation_loader = torch.utils.data.DataLoader(train_data, sampler=validation_sampler, batch_size=BATCH_SIZE)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
    print("Train-batches: ",len(train_loader))
    print("Valid-batches: ",len(validation_loader))
    print("Test-batches: ",len(test_loader))
    return train_loader, validation_loader, test_loader

train_loader, validation_loader, test_loader = init_data()




Train-batches:  2400
Valid-batches:  600
Test-batches:  500


In [3]:
#Define model
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        # Network will have input of 784 nodes and 3 hidden layers of 512, 256 and 128, 
        # with an output layer for class scores of size 10
        hidden_1_size = hl_slider1.value
        hidden_2_size = hl_slider2.value
        hidden_3_size = hl_slider3.value
        output_size = 10
        DROPOUT_PROB = dropout_slider.value
        
        self.fc1 = torch.nn.Linear(784, hidden_1_size)
        self.fc2 = torch.nn.Linear(hidden_1_size, hidden_2_size)
        self.fc3 = torch.nn.Linear(hidden_2_size, hidden_3_size)
        self.out = torch.nn.Linear(hidden_3_size, output_size)
        self.dropout = torch.nn.Dropout(p=DROPOUT_PROB)
        
        
    def forward(self, x):
        #Flatten
        x = x.view(-1, 784)
        # First hidden layer output
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        # Second hidden layer output
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        # Third hidden layer output
        x = torch.nn.functional.relu(self.fc3(x))
        x = self.dropout(x)
        # Last layer, i.e., the output
        x = self.out(x)
        
        return x
        
model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr_slider.value)

In [4]:
def init_model():
    model = Model()
    print(model)
    
def init_optimizer():
    optimizer = torch.optim.SGD(model.parameters(), lr = lr_slider.value)
    print(optimizer)

In [5]:
def change_batch_size_slider(change):
    batch_size_slider.value = change.new
    init_data()
    

def change_slider_1(change):
    hl_slider1.value = change.new
    init_model()
    
def change_slider_2(change):
    hl_slider2.value = change.new
    init_model()
    
def change_slider_3(change):
    hl_slider3.value = change.new
    init_model()

def change_dropout_slider(change):
    dropout_slider.value = change.new
    init_model()

def change_lr_slider(change):
    lr_slider.value = change.new
    init_optimizer()

def change_epoch_slider(change):
    epoch_slider.value = change.new
    

batch_size_slider.observe(change_batch_size_slider, names='value')
hl_slider1.observe(change_slider_1, names='value')
hl_slider2.observe(change_slider_2, names='value')
hl_slider3.observe(change_slider_3, names='value')
dropout_slider.observe(change_dropout_slider, names='value')
lr_slider.observe(change_lr_slider, names='value')
epoch_slider.observe(change_epoch_slider, names='value')

In [6]:

def train(b):
    print("Training now...")
    EPOCHS = epoch_slider.value
    valid_loss_min = np.Inf
    for epoch in range(EPOCHS):

        train_loss = 0.0
        validation_loss = 0.0

        model.train()
        for data, label in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()*data.size(0)

        model.eval()
        for data, label in validation_loader:
            output = model(data)
            loss = criterion(output, label)

            validation_loss += loss.item()*data.size(0)

        train_loss = train_loss / len(train_loader)
        validation_loss = validation_loss / len(validation_loader)

        print("Epoch: {} / Training Loss: {:.4f} / Validation Loss: {:.4f}".format(
            epoch+1,
            train_loss,
            validation_loss
            ))

        if validation_loss < valid_loss_min:
            print("Validation loss decreased from {:.4f} to {:.4f}. Saving the model.".format(
                valid_loss_min,
                validation_loss))
            torch.save(model.state_dict(), 'mnist_model.pt')
            valid_loss_min = validation_loss
    print("Training complete!")


In [7]:


def test(b):
    print("Start testing...")
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    test_loss = 0.0
    correct_guesses_for_class = [0 for i in range(10)]
    total_samples_per_class = [0 for i in range(10)]
    # Setup the model for testing
    model.eval()
    count = 0
    correct_pred = 0
    total_pred = 0
    for data, labels in test_loader:
        count = 1
        output = model(data)
        max_score, argmax_class_index = torch.max(output, 1)

        is_same_bool_array = argmax_class_index.eq(labels)
        correct_pred += int(is_same_bool_array.sum())
        total_pred += len(is_same_bool_array)


    print("The accuracy for overall test-set is: {:.4f}".format(100*(correct_pred/total_pred)))

## Train model

In [8]:
train_button = widgets.Button(description='TRAIN')
test_button = widgets.Button(description='TEST')
display(train_button)
display(test_button)
train_button.on_click(train)
test_button.on_click(test)

Button(description='TRAIN', style=ButtonStyle())

Button(description='TEST', style=ButtonStyle())